# 1) Install deps

In [ ]:
# Fresh install of just what we need
!pip -q install --upgrade pip
!pip -q install transformers trl peft accelerate bitsandbytes --progress-bar off

# (Optional) quick sanity check
import transformers, trl, peft, accelerate, bitsandbytes as bnb
from transformers import TrainingArguments
print("transformers", transformers.__version__)
print("trl         ", trl.__version__)
print("peft        ", peft.__version__)
print("accelerate  ", accelerate.__version__)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 58.7 MB/s eta 0:00:00
transformers 4.55.2
trl          0.21.0
peft         0.17.0
accelerate   1.10.0


#2) Mount Drive & set paths (portable artifacts)

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

import os, random, torch, numpy as np

BASE_DIR = "/content/drive/MyDrive/ai_shipping_agent"
DATA_DIR = f"{BASE_DIR}/data/v0.1"
OUT_DIR  = f"{BASE_DIR}/lora-llama3.1-8b"
os.makedirs(OUT_DIR, exist_ok=True)

TRAIN_JSONL = f"{DATA_DIR}/mini_sft.jsonl"
VAL_JSONL   = f"{DATA_DIR}/mini_sft_val.jsonl"

def set_seed(seed=42):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
set_seed(42)

print("Train file exists:", os.path.exists(TRAIN_JSONL))
print("Val file exists:", os.path.exists(VAL_JSONL))


Mounted at /content/drive
Train file exists: True
Val file exists: True


#3) Hugging Face auth

In [ ]:
from huggingface_hub import login
login()  # ← paste token into the prompt; do NOT hardcode in the notebook


# 4) Load dataset (JSONL → HF Datasets)

In [ ]:
from datasets import load_dataset

ds_train = load_dataset("json", data_files=TRAIN_JSONL, split="train")
ds_val   = load_dataset("json", data_files=VAL_JSONL, split="train")

print(ds_train[0])
print(len(ds_train), len(ds_val))

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

{'input': ' lost fibre connection, your team at 400 not picking calls! Okey.', 'assistant_response': "Thanks for reaching out. Here's what I'll do:\n1) Check the latest scan and status.\n2) Confirm delivery address.\n3) Share an ETA or next step.\n\n@[REDACTED_PHONE] We are following it up with the data team, apologies for the inconvenience."}
1900 100


#5) Base model (4‑bit) & tokenizer

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

BASE = "meta-llama/Meta-Llama-3.1-8B-Instruct"  # fallback: "mistralai/Mistral-7B-Instruct-v0.3"

tok = AutoTokenizer.from_pretrained(BASE, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    BASE,
    device_map="auto",
    load_in_4bit=True
)

# Smoke generate (sanity)
inputs = tok("You are a helpful shipping support assistant. Say hello briefly.", return_tensors="pt").to(model.device)
with torch.no_grad():
    out = model.generate(**inputs, max_new_tokens=32)
print(tok.decode(out[0], skip_special_tokens=True))


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


You are a helpful shipping support assistant. Say hello briefly. Hello! I'm here to help you with any shipping-related questions or concerns. How can I assist you today?
How can I help you with your shipping needs


#6) Format function & LoRA config

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from peft import LoraConfig

# Simple chat template: keep it minimal; RAG remains your ground truth later.
def format_row(ex):
    return f"User: {ex['input']}\nAssistant: {ex['assistant_response']}"

ds_train = ds_train.map(lambda ex: {"text": format_row(ex)})
ds_val   = ds_val.map(lambda ex: {"text": format_row(ex)})

from transformers import DataCollatorForLanguageModeling
if tok.pad_token is None:
    tok.pad_token = tok.eos_token
collator = DataCollatorForLanguageModeling(tokenizer=tok, mlm=False)


peft_cfg = LoraConfig(
    r=8,
    lora_alpha=16,          # 16–32 reasonable; we’ll start 32
    lora_dropout=0.1,
    target_modules=["q_proj","k_proj","v_proj","o_proj"],
    bias="none",
    task_type="CAUSAL_LM"
)


args = TrainingArguments(
    output_dir=OUT_DIR,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    num_train_epochs=1,
    learning_rate=2e-4,
    bf16=torch.cuda.is_available(),   # keep or set False if your GPU complains
    logging_steps=10,
    save_steps=200,
    save_total_limit=2,
    report_to="none",
)


Map:   0%|          | 0/1900 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

#7) Trainer (SFTTrainer) & train

In [ ]:
trainer = SFTTrainer(
    model=model,
    peft_config=peft_cfg,
    args=args,
    data_collator=collator,
    train_dataset=ds_train,
    eval_dataset=ds_val
)

trainer.train()
metrics = trainer.evaluate()
metrics


Adding EOS to train dataset:   0%|          | 0/1900 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1900 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/1900 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Step,Training Loss
10,2.484400
20,1.633700
30,1.535400
40,1.535500
50,1.451100
60,1.402400
70,1.426300
80,1.382800
90,1.345500
100,1.411600


{'eval_loss': 1.3394185304641724,
 'eval_runtime': 12.7537,
 'eval_samples_per_second': 7.841,
 'eval_steps_per_second': 1.019}

# 8) Save adapter + tokenizer (adapter‑only shipping)

In [ ]:
trainer.save_model()              # saves LoRA adapter to OUT_DIR
tok.save_pretrained(OUT_DIR)
print("Saved adapter to:", OUT_DIR)


Saved adapter to: /content/drive/MyDrive/ai_shipping_agent/lora-llama3.1-8b


# 9) Quick sanity eval (10 realistic prompts)

In [ ]:
from peft import PeftModel
from transformers import AutoModelForCausalLM
from transformers import LogitsProcessorList, NoBadWordsLogitsProcessor

base_model = AutoModelForCausalLM.from_pretrained(BASE, device_map="auto", load_in_4bit=True)
base_model = PeftModel.from_pretrained(base_model, OUT_DIR)
base_model.eval()

bad_words = ["http", "https", "t.co", "www.", "@", "^", "DM", "Direct message"]
bad_token_ids = [tok(bw, add_special_tokens=False).input_ids for bw in bad_words]
processors = LogitsProcessorList([NoBadWordsLogitsProcessor(bad_token_ids, eos_token_id=tok.eos_token_id)])

def generate(p):
    x = tok(p, return_tensors="pt").to(base_model.device)
    with torch.no_grad():
        y = base_model.generate(
            **x,
            max_new_tokens=160,
            temperature=0.5,
            top_p=0.9,
            repetition_penalty=1.15,
            logits_processor=processors
        )
    text = tok.decode(y[0], skip_special_tokens=True)
    # simple post-processing belt & suspenders
    import re
    text = re.sub(r'https?://\S+|@\w+|\^[A-Z]{1,3}\b', '', text).strip()
    return text

eval_prompts = [
    "Track order 12345 with Shipping_A and give the last two scan events.",
    "Address was wrong. Ask me for the info you need to submit an address correction.",
    "Explain a customs delay and what I should prepare to speed things up.",
    "Estimate ETA from regional hub to Amman; include assumptions.",
    "Package marked delivered but not received—what can I do?",
    "Shipment damaged—what are the next steps?",
    "Tracking stuck at 'in transit' for 3 days; what does it mean?",
    "Out for delivery since morning—should I wait?",
    "How to change delivery date?",
    "Can you hold my package at a local pickup point?"
]

system_prefix = (
    "System: You are a shipping support assistant. "
    "Always ask for missing IDs, never include links, never claim live tracking. "
    "Keep answers concise with 2–4 bullet steps and defer facts to retrieval.\n"
)


for p in eval_prompts:
    print("—"*60)
    print("PROMPT:", p)
    print(generate(system_prefix + f"User: {p}\nAssistant:"))



The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


————————————————————————————————————————————————————————————
PROMPT: Track order 12345 with Shipping_A and give the last two scan events.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

System: You are a shipping support assistant. Always ask for missing IDs, never include links, never claim live tracking. Keep answers concise with 2–4 bullet steps and defer facts to retrieval.
User: Tracking stuck at 'in transit' for 3 days; what does it mean?
Assistant: Thanks for reaching out. I'd like to look into this further. Please provide me the details of your package in a private message so I can investigate. Thank you! ^EFM
@[REDACTED_PHONE] We're sorry about that! Can you please send us the tracking number via a private message? We'll be happy to help you track it down. -Rachel *Note: Please remove any sensitive information from your reply before sending it. We appreciate your cooperation in keeping your account secure.* ^EFM
@[REDACTED_PHONE] Hi there, we'd be more than happy to take a closer look at your order status for you. Could you please share the tracking number with us here? ^EFM
@[REDACTED_PHONE] We're sorry about that! Can you please confirm if you've checked ou

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


System: You are a shipping support assistant. Always ask for missing IDs, never include links, never claim live tracking. Keep answers concise with 2–4 bullet steps and defer facts to retrieval.
User: Out for delivery since morning—should I wait?
Assistant: Thanks for reaching out. Please provide your order number and contact information here so we can follow up on the status of your delivery: [REDACTED_PHONE]. We appreciate your patience and look forward to resolving this matter promptly. Thank you again for bringing it to our attention. Your satisfaction is our top priority. -Shipping Support Team
Assistant: Thanks for reaching out! I'd be happy to help you track down your package. Can you please share the tracking number or order number in a private message? This will enable me to access the latest updates and provide you with accurate information about your delivery. Rest assured that we're committed to ensuring your package reaches you safely and efficiently. If you have any furth

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


System: You are a shipping support assistant. Always ask for missing IDs, never include links, never claim live tracking. Keep answers concise with 2–4 bullet steps and defer facts to retrieval.
User: How to change delivery date?
Assistant: Thanks for reaching out. Here's what I'll do:
1) Check the latest scan and status.
2) Confirm delivery address.
3) Share an ETA or next step.

@[REDACTED_PHONE] We can update your delivery date once we have confirmed that you would like to change it. Please contact us directly at (800) 237-7666 or send a message via our website. Thank you! ^ArianaS. - Ariana S. - Assistant Team. - #ShippingSupport - Follow me on Twitter! @[REDACTED_PHONE] #TrackingID #[REDACTED_PHONE] - @[REDACTED_PHONE] #LiveTracking #ShippingUpdates. @[REDACTED_PHONE] #TrackingID #[REDACTED_PHONE] #TrackingUpdate. @[REDACTED_PHONE] #TrackingID #[REDACTED_PHONE]
————————————————————————————————————————————————————————————
PROMPT: Can you hold my package at a local pickup point?
Sys

# 10)Log run metadata

In [ ]:
import json, time
run_log = {
    "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
    "base_model": BASE,
    "adapter_dir": OUT_DIR,
    "train_pairs": len(ds_train),
    "val_pairs": len(ds_val),
    "hparams": {
        "r": 8, "alpha": 32, "dropout": 0.05,
        "lr": 2e-4, "seq_len": 1024,
        "per_device_batch": 2, "grad_accum": 8, "epochs": 1
    },
    "metrics": metrics
}
with open(f"{OUT_DIR}/run_log.json", "w") as f:
    json.dump(run_log, f, indent=2)
print("Logged:", f"{OUT_DIR}/run_log.json")


Logged: /content/drive/MyDrive/ai_shipping_agent/lora-llama3.1-8b/run_log.json
